In [1]:
!/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
!pip install yfinance pandas numpy requests openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Define start and end dates
end_date = datetime.today()
start_date = end_date - timedelta(days=365 * 10)

# Download gold data using the GLD ETF
gold_data = yf.download('GLD', start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
gold_data = gold_data.reset_index()

# Normalize date column to remove time info
gold_data['Date'] = pd.to_datetime(gold_data['Date']).dt.normalize()
gold_data = gold_data.sort_values(by='Date').reset_index(drop=True)

# Function to get the next trading day if 5th is not available
def get_next_trading_day(date, trading_days):
    while date not in trading_days and date <= trading_days[-1]:
        date += timedelta(days=1)
    return date if date in trading_days else None

# Generate 5th of each month
fifth_days = pd.date_range(start=start_date, end=end_date, freq='MS') + pd.DateOffset(days=4)
fifth_days = [d.normalize() for d in fifth_days]

# List of available trading days (normalized)
trading_days = list(gold_data['Date'])

# # Get adjusted buy dates
buy_dates = [get_next_trading_day(d, trading_days) for d in fifth_days]
buy_dates = [d for d in buy_dates if d is not None]

# Buy on the last trading day of each month
gold_data['YearMonth'] = gold_data['Date'].dt.to_period('M')
last_days = gold_data.groupby('YearMonth')['Date'].max().tolist()
buy_dates = last_days

# Handle case where buy_dates is still empty
if not buy_dates:
    raise ValueError("No valid buy dates found. Check if gold_data has correct trading days.")

# Initialize investment tracking
initial_investment = 5000
year_start = buy_dates[0].year
investment_log = []

for date in buy_dates:
    years_passed = date.year - year_start
    current_investment = initial_investment * (1.10 ** years_passed)

    price_row = gold_data[gold_data['Date'] == date]
    if not price_row.empty:
        # Fix the price extraction in main loop
        price = float(price_row['Close'].iloc[0])
        gold_units = current_investment / price
        investment_log.append({
            'Date': date,
            'Investment': round(current_investment, 2),
            'Price': round(price, 2),
            'Gold Units': gold_units
        })

# Get latest available price for gold
latest_price = float(price_row['Close'].iloc[-1])

# Final dataframe
investment_df = pd.DataFrame(investment_log)
investment_df2 = pd.DataFrame(investment_log)

# Final gold price
final_price = gold_data.iloc[-1]['Close']

# Update Current Value column
investment_df2['Current Value'] = investment_df2['Gold Units'] * latest_price

# Calculate totals
total_invested = investment_df2['Investment'].sum()
current_value = investment_df2['Current Value'].sum()
total_return = current_value - total_invested
roi = (total_return / total_invested) * 100

# Output
print(investment_df)
print(f"\nTotal Invested: ₹{total_invested:,.2f}")
print(f"Current Value: ₹{current_value:,.2f}")
print(f"Total Return: ₹{total_return:,.2f}")
print(f"ROI: {roi:.2f}%")


/Users/nganvit/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

          Date  Investment   Price  Gold Units
0   2015-04-30     5000.00  113.47   44.064510
1   2015-05-29     5000.00  114.10   43.821210
2   2015-06-30     5000.00  112.37   44.495861
3   2015-07-31     5000.00  104.93   47.650815
4   2015-08-31     5000.00  108.82   45.947436
..         ...         ...     ...         ...
116 2024-12-31    11789.74  242.13   48.691770
117 2025-01-31    12968.71  258.56   50.157458
118 2025-02-28    12968.71  263.27   49.260124
119 2025-03-31    12968.71  288.14   45.008370
120 2025-04-10    12968.71  292.55   44.329717

[121 rows x 4 columns]

Total Invested: ₹993,120.40
Current Value: ₹1,862,714.69
Total Return: ₹869,594.29
ROI: 87.56%



/var/folders/m7/8r4fmcds69q3n8s36mmpmpjr0000gn/T/ipykernel_11623/3717499796.py:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(price_row['Close'].iloc[0])
/var/folders/m7/8r4fmcds69q3n8s36mmpmpjr0000gn/T/ipykernel_11623/3717499796.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  latest_price = float(price_row['Close'].iloc[-1])


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Simulate monthly real estate price data (₹/sq.ft.)
start_date = datetime(2015, 5, 5)
end_date = datetime(2025, 4, 10)

dates = pd.date_range(start=start_date, end=end_date, freq='MS')  # Month start
initial_price = 4000  # ₹/sq.ft. in 2015
annual_appreciation_rate = 0.06  # 6% annual growth

# Generate price data with compounded monthly growth
monthly_growth = (1 + annual_appreciation_rate) ** (1 / 12) - 1
prices = [initial_price * (1 + monthly_growth) ** i for i in range(len(dates))]
real_estate_df = pd.DataFrame({'Date': dates, 'Price_per_sqft': prices})

# Choose 5th of every month as buying date
buy_dates = pd.date_range(start=start_date, end=end_date, freq='MS') + pd.Timedelta(days=4)
buy_dates = buy_dates[buy_dates <= end_date]

# Investment settings
initial_investment = 5000
investment_log = []

year_start = buy_dates[0].year
current_investment = initial_investment

for date in buy_dates:
    # Adjust investment amount by 10% annually
    if date.year > year_start:
        current_investment *= 1.10
        year_start = date.year

    price_row = real_estate_df[real_estate_df['Date'] == date.replace(day=1)]
    if price_row.empty:
        continue

    price_per_sqft = float(price_row['Price_per_sqft'].iloc[0])
    area_bought = current_investment / price_per_sqft

    investment_log.append({
        'Date': date.date(),
        'Investment': round(current_investment, 2),
        'Price_per_sqft': round(price_per_sqft, 2),
        'Area_sqft': round(area_bought, 4)
    })

# Create DataFrame
investment_df = pd.DataFrame(investment_log)

# Use the latest price for current value
latest_price = float(real_estate_df['Price_per_sqft'].iloc[-1])
investment_df['Current Value'] = investment_df['Area_sqft'] * latest_price

# Final stats
total_invested = investment_df['Investment'].sum()
current_value = investment_df['Current Value'].sum()
total_return = current_value - total_invested
roi = (total_return / total_invested) * 100

# Output
print(investment_df.tail())
print(f"\nTotal Invested: ₹{total_invested:,.2f}")
print(f"Current Value: ₹{current_value:,.2f}")
print(f"Total Return: ₹{total_return:,.2f}")
print(f"ROI: {roi:.2f}%")


           Date  Investment  Price_per_sqft  Area_sqft  Current Value
114  2024-12-05    11789.74         6957.70     1.6945   12021.054689
115  2025-01-05    12968.71         6991.57     1.8549   13158.958006
116  2025-02-05    12968.71         7025.60     1.8459   13095.110563
117  2025-03-05    12968.71         7059.80     1.8370   13031.972536
118  2025-04-05    12968.71         7094.16     1.8281   12968.834509

Total Invested: ₹983,120.40
Current Value: ₹1,268,638.05
Total Return: ₹285,517.65
ROI: 29.04%


In [4]:
import pandas as pd

# Load the data
hpi_data = pd.read_excel('/Users/nganvit/Downloads/HPI@Assessment Prices_Index.xlsx', header=0)

# Melt the data (unpivot)
melted = hpi_data.melt(id_vars='City', var_name='Date', value_name='Index')

# Clean up the date column and convert to datetime
melted['Date'] = melted['Date'].str.strip().str.replace('--', '-', regex=False).str.replace('  ', ' ', regex=False)
melted['Date'] = pd.to_datetime(melted['Date'], errors='coerce', format='%b- %Y')

# Drop rows with invalid dates or missing values
melted.dropna(subset=['Date', 'Index'], inplace=True)

# Choose a city (e.g., Mumbai)
city_data = melted[melted['City'].str.lower() == 'mumbai'].sort_values('Date')

# Set Date as index
city_data.set_index('Date', inplace=True)

# Filter for 5th day of every month (or as close as possible)
investment_dates = pd.date_range(start='2015-05-05', end='2025-04-10', freq='MS') + pd.Timedelta(days=4)

# Simulate investment
investment_log = []
initial_investment = 5000
start_year = investment_dates[0].year

for date in investment_dates:
    # Find the index of the closest available date
    idx = city_data.index.get_indexer([date], method='nearest')[0]
    closest_date = city_data.index[idx]
    price = city_data.loc[closest_date, 'Index']
    
    # Adjust investment amount annually
    years_passed = date.year - start_year
    current_investment = initial_investment * (1.1 ** years_passed)
    
    units = current_investment / price
    investment_log.append({
        'Date': closest_date,
        'Investment': round(current_investment, 2),
        'Index': round(price, 2),
        'Units': units
    })

# Create DataFrame
df = pd.DataFrame(investment_log)

# Calculate final value
latest_index = city_data['Index'].iloc[-1]
df['Current Value'] = df['Units'] * latest_index

total_invested = df['Investment'].sum()
total_value = df['Current Value'].sum()
roi = ((total_value - total_invested) / total_invested) * 100

print(df.tail())
print(f"\nTotal Invested: ₹{total_invested:,.2f}")
print(f"Current Value: ₹{total_value:,.2f}")
print(f"Total Return: ₹{(total_value - total_invested):,.2f}")
print(f"ROI: {roi:.2f}%")


          Date  Investment   Index       Units  Current Value
114 2024-12-01    11789.74  119.76   98.444710   11789.738455
115 2024-12-01    12968.71  119.76  108.289181   12968.712301
116 2024-12-01    12968.71  119.76  108.289181   12968.712301
117 2024-12-01    12968.71  119.76  108.289181   12968.712301
118 2024-12-01    12968.71  119.76  108.289181   12968.712301

Total Invested: ₹983,120.40
Current Value: ₹1,081,122.18
Total Return: ₹98,001.78
ROI: 9.97%


In [5]:
import pandas as pd
from datetime import datetime

# Load the HPI dataset
file_path = '/Users/nganvit/Downloads/HPI@Assessment Prices_Index.xlsx'
hpi_raw = pd.read_excel(file_path)

# Reshape data to long format: 'City', 'Date', 'Index'
hpi_long = hpi_raw.melt(id_vars='City', var_name='Date', value_name='Index')

# Standardize the date column
hpi_long['Date'] = hpi_long['Date'].str.replace('--', '-').str.replace('  ', ' ').str.strip()
hpi_long['Date'] = pd.to_datetime(hpi_long['Date'], format='%b-%Y', errors='coerce')

# Drop any rows with invalid dates
hpi_long = hpi_long.dropna(subset=['Date'])

# Set start and end dates for investment simulation
start_date = pd.to_datetime('2015-05-05')
end_date = pd.to_datetime('2025-04-10')

# Generate investment dates: 5th of every month
investment_dates = pd.date_range(start=start_date, end=end_date, freq='MS') + pd.Timedelta(days=24)

# Initialize master log
all_city_results = []

# Buy on the last trading day of each month
hpi_long['YearMonth'] = hpi_long['Date'].dt.to_period('M')
last_days = hpi_long.groupby('YearMonth')['Date'].max().tolist()
buy_dates = last_days

# Loop over each city
for city in hpi_long['City'].unique():
    city_data = hpi_long[hpi_long['City'] == city].copy()
    city_data.set_index('Date', inplace=True)
    city_data = city_data.sort_index()

    investment_log = []
    initial_investment = 5000
    # start_year = investment_dates[0].year
    start_year = buy_dates[0].year

    # for date in investment_dates:
    for date in buy_dates:
        # Skip if no data exists
        if city_data.empty: continue

        # Get nearest date
        try:
            idx = city_data.index.get_indexer([date], method='nearest')[0]
            closest_date = city_data.index[idx]
        except:
            continue

        price = city_data.loc[closest_date, 'Index']

        # Adjust investment amount annually
        years_passed = date.year - start_year
        current_investment = initial_investment * (1.1 ** years_passed)

        units = current_investment / price
        investment_log.append({
            'City': city,
            'Date': closest_date,
            'Investment': round(current_investment, 2),
            'Index': round(price, 2),
            'Units': units
        })

    # Final calculations
    city_df = pd.DataFrame(investment_log)
    if not city_df.empty:
        total_invested = city_df['Investment'].sum()
        total_units = city_df['Units'].sum()
        latest_index = city_df.iloc[-1]['Index']
        current_value = total_units * latest_index
        roi = ((current_value - total_invested) / total_invested) * 100

        city_summary = {
            'City': city,
            'Total Invested (₹)': round(total_invested, 2),
            'Current Value (₹)': round(current_value, 2),
            'ROI (%)': round(roi, 2)
        }
        all_city_results.append(city_summary)

# Compile all results
summary_df = pd.DataFrame(all_city_results).sort_values(by='ROI (%)', ascending=False)

# Display summary
print(summary_df)


                                 City  Total Invested (₹)  Current Value (₹)  \
49                              Vizag           141259.65          171810.20   
42                             Ranchi           141259.65          169639.87   
39                               Pune           141259.65          167770.39   
19                          Hyderabad           141259.65          167501.11   
0                           Ahmedabad           141259.65          167099.83   
6                              Chakan           141259.65          166544.36   
43                              Surat           141259.65          166360.25   
1                           Bengaluru           141259.65          166025.48   
22                   Kalyan Dombivali           141259.65          165119.59   
30                             Mumbai           141259.65          164979.26   
34                   New Town Kolkata           141259.65          164796.82   
44                              Thane   

In [6]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# --- Config ---
ticker = "TCS.NS"  # You can change this to any valid ticker
start_date = "2015-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

# Download historical stock data
data = yf.download(ticker, start=start_date, end=end_date)
data.reset_index(inplace=True)

# Generate all 5th-of-month dates for past 10 years
buy_dates = pd.date_range(start=start_date, end=end_date, freq='MS') + pd.Timedelta(days=4)
buy_dates = [date for date in buy_dates if date in data['Date'].values]

# Ensure all dates are trading days
valid_dates = []
for date in buy_dates:
    match = data[data['Date'] == date]
    if not match.empty:
        valid_dates.append(date)
    else:
        # Use next available trading day
        future_date = date
        while True:
            future_date += timedelta(days=1)
            if future_date in data['Date'].values:
                valid_dates.append(future_date)
                break

# Initialize investment tracking
initial_investment = 5000
investment_log = []
year_start = valid_dates[0].year

for date in valid_dates:
    if date.year > year_start:
        initial_investment *= 1.10
        year_start = date.year
    
    price = float(data[data['Date'] == date]['Close'].iloc[0])
    stock_units = initial_investment / price
    
    investment_log.append({
        'Date': date.strftime('%Y-%m-%d'),
        'Investment': round(initial_investment, 2),
        'Price': round(price, 2),
        'Stock Units': stock_units
    })

# Create DataFrame
investment_df = pd.DataFrame(investment_log)

# Calculate Current Value
latest_price = float(data['Close'].iloc[-1])
investment_df['Current Value'] = investment_df['Stock Units'] * latest_price

# Totals
total_invested = investment_df['Investment'].sum()
current_value = investment_df['Current Value'].sum()
total_return = current_value - total_invested
roi = (total_return / total_invested) * 100

# Display results
print(investment_df.tail())
print(f"\nTotal Invested: ₹{total_invested:,.2f}")
print(f"Current Value: ₹{current_value:,.2f}")
print(f"Total Return: ₹{total_return:,.2f}")
print(f"ROI: {roi:.2f}%")


[*********************100%***********************]  1 of 1 completed

          Date  Investment    Price  Stock Units  Current Value
79  2024-09-05    11789.74  4454.44     2.646738    8592.898263
80  2024-11-05    11789.74  3961.91     2.975772    9661.142899
81  2024-12-05    11789.74  4453.44     2.647335    8594.837183
82  2025-02-05    12968.71  4091.10     3.169982   10291.662784
83  2025-03-05    12968.71  3547.05     3.656197   11870.208213

Total Invested: ₹682,527.80
Current Value: ₹1,089,606.10
Total Return: ₹407,078.30
ROI: 59.64%



/var/folders/m7/8r4fmcds69q3n8s36mmpmpjr0000gn/T/ipykernel_11623/1490913816.py:43: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(data[data['Date'] == date]['Close'].iloc[0])
/var/folders/m7/8r4fmcds69q3n8s36mmpmpjr0000gn/T/ipykernel_11623/1490913816.py:57: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  latest_price = float(data['Close'].iloc[-1])


In [7]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Define large-cap Indian stocks
large_cap_tickers = [
    'RELIANCE.NS', 'HDFCBANK.NS', 'TCS.NS', 'INFY.NS', 'ICICIBANK.NS',
    'HINDUNILVR.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'ITC.NS', 'BAJFINANCE.NS',
    'LT.NS', 'KOTAKBANK.NS', 'AXISBANK.NS', 'MARUTI.NS', 'SUNPHARMA.NS',
    'HCLTECH.NS', 'M&M.NS', 'TATASTEEL.NS', 'NTPC.NS', 'POWERGRID.NS',
    'ULTRACEMCO.NS', 'TITAN.NS', 'WIPRO.NS', 'ASIANPAINT.NS', 'NESTLEIND.NS',
    'HDFCLIFE.NS', 'DRREDDY.NS', 'JSWSTEEL.NS', 'DIVISLAB.NS', 'BAJAJFINSV.NS',
    'TECHM.NS', 'GRASIM.NS', 'ADANIPORTS.NS', 'CIPLA.NS', 'SBILIFE.NS',
    'BRITANNIA.NS', 'HEROMOTOCO.NS', 'COALINDIA.NS', 'ONGC.NS', 'HINDALCO.NS',
    'BPCL.NS', 'APOLLOHOSP.NS', 'EICHERMOT.NS', 'UPL.NS', 'SHREECEM.NS',
    'TATAMOTORS.NS', 'INDUSINDBK.NS', 'BAJAJ-AUTO.NS', 'IOC.NS'
]

# Date range: last 10 years
end_date = datetime.today()
start_date = end_date - timedelta(days=365 * 10)

# Investment schedule: 5th of every month
buy_dates = pd.date_range(start=start_date, end=end_date, freq='MS') + pd.DateOffset(days=14)
buy_dates = buy_dates[buy_dates <= end_date]

# Fetch historical prices
data = yf.download(large_cap_tickers, start=start_date, end=end_date, interval='1d', group_by='ticker', auto_adjust=True)

# Investment tracking
investment_summary = []
initial_investment = 5000
increase_rate = 0.10

for ticker in large_cap_tickers:
    total_units = 0
    total_invested = 0
    investment = initial_investment
    prev_year = buy_dates[0].year

    for date in buy_dates:
        if date.year != prev_year:
            investment *= (1 + increase_rate)
            prev_year = date.year

        try:
            price = data[ticker].loc[date.strftime('%Y-%m-%d')]['Close']
            if not pd.isna(price):
                units = investment / price
                total_units += units
                total_invested += investment
        except KeyError:
            continue

    try:
        latest_price = data[ticker]['Close'].dropna()[-1]
        current_value = total_units * latest_price
        roi = ((current_value - total_invested) / total_invested) * 100
    except Exception:
        current_value = 0
        roi = -100

    investment_summary.append({
        'Stock': ticker.replace('.NS', ''),
        'Total Invested (₹)': round(total_invested, 2),
        'Current Value (₹)': round(current_value, 2),
        'ROI (%)': round(roi, 2)
    })

# Create and show summary
summary_df = pd.DataFrame(investment_summary)
summary_df.sort_values(by='ROI (%)', ascending=False, inplace=True)
print(summary_df.reset_index(drop=True))


[*********************100%***********************]  49 of 49 completed
/var/folders/m7/8r4fmcds69q3n8s36mmpmpjr0000gn/T/ipykernel_11623/217413636.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_price = data[ticker]['Close'].dropna()[-1]


         Stock  Total Invested (₹)  Current Value (₹)  ROI (%)
0    TATASTEEL           604367.23         2937910.05   386.11
1   BAJFINANCE           604367.23         2330990.86   285.69
2   BHARTIARTL           604367.23         2100492.38   247.55
3     JSWSTEEL           604367.23         2025604.35   235.16
4   APOLLOHOSP           604367.23         2009470.69   232.49
5          M&M           604367.23         1906648.33   215.48
6        TITAN           604367.23         1899792.47   214.34
7         NTPC           604367.23         1882550.65   211.49
8   BAJAJFINSV           604367.23         1847378.94   205.67
9    POWERGRID           604367.23         1807634.13   199.10
10   ICICIBANK           604367.23         1769998.88   192.87
11    DIVISLAB           604367.23         1743965.29   188.56
12   COALINDIA           604367.23         1500962.79   148.35
13      GRASIM           604367.23         1500690.79   148.31
14    HINDALCO           604367.23         1475283.59  

In [8]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# --- Config ---
ticker = "NIFTYBEES.NS"  # ETF that tracks Nifty 50
start_date = "2015-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

# Download ETF data
data = yf.download(ticker, start=start_date, end=end_date)
data.reset_index(inplace=True)

# Generate 5th-of-month buy dates
buy_dates = pd.date_range(start=start_date, end=end_date, freq='MS') + pd.Timedelta(days=4)
buy_dates = [d for d in buy_dates if d <= pd.to_datetime(end_date)]

# Adjust to actual trading days
valid_dates = []
for d in buy_dates:
    row = data[data['Date'] == d]
    if not row.empty:
        valid_dates.append(d)
    else:
        temp = d
        while True:
            temp += timedelta(days=1)
            row = data[data['Date'] == temp]
            if not row.empty:
                valid_dates.append(temp)
                break

# Simulate investments
initial_investment = 5000
investment_log = []
year_start = valid_dates[0].year

for date in valid_dates:
    if date.year > year_start:
        initial_investment *= 1.10
        year_start = date.year

    price = float(data[data['Date'] == date]['Close'].iloc[0])
    units = initial_investment / price

    investment_log.append({
        'Date': date.strftime('%Y-%m-%d'),
        'Investment': round(initial_investment, 2),
        'Price': round(price, 2),
        'Units Bought': units
    })

# Create DataFrame
df = pd.DataFrame(investment_log)

# Use latest price to calculate current value
latest_price = float(data['Close'].iloc[-1])
df['Current Value'] = df['Units Bought'] * latest_price

# Totals
total_invested = df['Investment'].sum()
current_value = df['Current Value'].sum()
total_return = current_value - total_invested
roi = (total_return / total_invested) * 100

# Results
print(df.tail())
print(f"\nTotal Invested: ₹{total_invested:,.2f}")
print(f"Current Value: ₹{current_value:,.2f}")
print(f"Total Return: ₹{total_return:,.2f}")
print(f"ROI: {roi:.2f}%")


[*********************100%***********************]  1 of 1 completed

           Date  Investment   Price  Units Bought  Current Value
119  2024-12-05    11789.74  275.92     42.728827   10726.217331
120  2025-01-06    12968.71  264.48     49.034754   12309.194259
121  2025-02-05    12968.71  265.25     48.892412   12273.462141
122  2025-03-05    12968.71  250.48     51.775441   12997.188951
123  2025-04-07    12968.71  249.10     52.062272   13069.192104

Total Invested: ₹1,008,120.40
Current Value: ₹1,798,802.62
Total Return: ₹790,682.22
ROI: 78.43%



/var/folders/m7/8r4fmcds69q3n8s36mmpmpjr0000gn/T/ipykernel_11623/2189330222.py:43: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(data[data['Date'] == date]['Close'].iloc[0])
/var/folders/m7/8r4fmcds69q3n8s36mmpmpjr0000gn/T/ipykernel_11623/2189330222.py:57: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  latest_price = float(data['Close'].iloc[-1])


In [9]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# --- Config ---
ticker = "NIFTYBEES.NS"
start_date = "2015-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

# Download historical data
data = yf.download(ticker, start=start_date, end=end_date)
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data['Date'])

# --- Generate safe buy dates (5th or nearest trading day) ---
buy_dates = pd.date_range(start=start_date, end=end_date, freq='MS') + pd.Timedelta(days=4)
buy_dates = [d for d in buy_dates if d <= data['Date'].max()]  # filter within available data

# Buy on the last trading day of each month
gold_data['YearMonth'] = gold_data['Date'].dt.to_period('M')
last_days = gold_data.groupby('YearMonth')['Date'].max().tolist()
buy_dates = last_days

valid_dates = []
for d in buy_dates:
    # If exact 5th is a trading day, use it
    if d in set(data['Date']):
        valid_dates.append(d)
    else:
        # Find next available trading day (max 7 days ahead to avoid infinite loops)
        for offset in range(1, 8):
            next_day = d + timedelta(days=offset)
            if next_day in set(data['Date']):
                valid_dates.append(next_day)
                break  # Exit loop once found

# --- Simulate Investments ---
initial_investment = 5000
investment_log = []
year_start = valid_dates[0].year
current_investment = initial_investment

for date in valid_dates:
    # Increase investment yearly
    if date.year > year_start:
        current_investment *= 1.10
        year_start = date.year

    price = float(data.loc[data['Date'] == date, 'Close'].iloc[0])
    units = current_investment / price

    investment_log.append({
        'Date': date.strftime('%Y-%m-%d'),
        'Investment': round(current_investment, 2),
        'Price': round(price, 2),
        'Units Bought': units
    })

# --- Final DataFrame ---
df = pd.DataFrame(investment_log)
latest_price = float(data['Close'].iloc[-1])
df['Current Value'] = df['Units Bought'] * latest_price

# --- Summary ---
total_invested = df['Investment'].sum()
current_value = df['Current Value'].sum()
total_return = current_value - total_invested
roi = (total_return / total_invested) * 100

print(df.tail())
print(f"\nTotal Invested: ₹{total_invested:,.2f}")
print(f"Current Value: ₹{current_value:,.2f}")
print(f"Total Return: ₹{total_return:,.2f}")
print(f"ROI: {roi:.2f}%")


[*********************100%***********************]  1 of 1 completed
/var/folders/m7/8r4fmcds69q3n8s36mmpmpjr0000gn/T/ipykernel_11623/1006101503.py:49: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(data.loc[data['Date'] == date, 'Close'].iloc[0])


           Date  Investment   Price  Units Bought  Current Value
115  2024-11-29    11789.74  269.88     43.685113   10966.273812
116  2024-12-31    11789.74  264.47     44.578736   11190.600130
117  2025-01-31    12968.71  263.00     49.310693   12378.463243
118  2025-02-28    12968.71  248.45     52.198480   13103.384476
119  2025-04-01    12968.71  259.60     49.956518   12540.584578

Total Invested: ₹980,151.69
Current Value: ₹1,732,266.99
Total Return: ₹752,115.30
ROI: 76.73%


/var/folders/m7/8r4fmcds69q3n8s36mmpmpjr0000gn/T/ipykernel_11623/1006101503.py:61: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  latest_price = float(data['Close'].iloc[-1])
